# Disciplina: Introdução à Inteligência Artificial
Instituto Federal do Norte de Minas Gerais Campus Montes Claros <br/>
Curso: Ciência da Computação <br/>
Aluno: David Walter Jansen <br/>

**Experimento 1: um algoritmo genético simples, com cruzamento**
<ol>
<li> Geração da população inicial, de tamanho N, de forma aleatória, entre -5.12 e 5.12. Sugestão N=20 </li>
<li> Seleção por torneio 2 a 2. Lembrando que os indivíduos selecionados podem ser repetidos. </li>
<li> Realizar N/2 cruzamentos com 90% de chance para os indivíduos campeões do torneio. Cada cruzamento produz 2 filhos. E cada filho tem um percentual alpha de participação de um pai, e 1-a lpha de participação do pai 2. </li>
<li> Realizar a mutação, sendo que, a taxa de mutação seja de 10% (por exemplo, 10 a cada 100 indivíduos devem sofrer mutação).</li>
- Da população selecionada no item 3, você deve escolher 10% deles e realizar uma perturbação de + ou - 5% do valor do indivíduo. Por exemplo, selecione um indivíduo, ele deve contar x_1 e x_2: <br/>
-> Selecione qual variável/dimensão você deseja alterar <br/>
-> Selecione qual operação você irá realizar +5% ou -5% <br/>

<li> Entre os pais e filhos, escolha os novos N indivíduos, que irão para a próxima geração</li>
<li> Exiba a cada geração o melhor indivíduo (x e f(x))</li>
<li> Repita 2 a 6 até atingir o número máximo de gerações. Sugestão: 100 gerações.</li>
</ol>
Depois do algoritmo pronto, execute-o 30 vezes. Para cada execução salve o melhor indivíduo ao final das gerações. Portanto, serão 30 indivíduos diferentes

In [54]:
import numpy as np
import random
import math

In [55]:
num_ger = 100 #numero máximo de gerações que AG executará
num_pop = 20  #haverá 20 individuos na populacao
chancem = 0.1 #10% dos individuos selecionados sofrerão mutação
chancec = 0.9 #90% dos individuos selecionados sofrerão cruzamento
num_dim=2
xmin=-5.12
xmax=5.12

In [56]:
#define a função para gerar a pop inicial de forma aleatória
def gera_pop_inicial(num_pop, n_dim,xmin, xmax):
    return np.random.uniform(xmin,xmax,(num_pop,num_dim))

In [57]:
def inverte(fx):
    hx = min(fx);
    desl = 10**-0.2 - hx;
    return 1/( fx + desl);

def rastrigin(X, **kwargs):
    
    A = kwargs.get('A')
    [row, col] = X.shape
    fx= np.zeros(row)
    for i in range(row):
        for j in range(col):
            fx[i] += X[i,j]**2 - A * np.cos(2 * math.pi * X[i,j])
    #print(X)
    return A*col + fx

In [58]:
def popTorneio(ind, X):
    [row, col] = X.shape
    Xt = np.zeros((row,col))
    
    for i in range(row):
        Xt[i] = X[ind[i].astype(int)]
        
    return Xt

In [59]:
def torneio(fx):
    ind = np.zeros(len(fx))
    for i in range(len(fx)):
        #sorteia dois individuos e seleciona o melhor com relação ao fitness, armazena o indice do vendedor
        i1 = random.randint(0,len(fx)-1)
        i2 = random.randint(0,len(fx)-1)
        #print(i1,i2)
        if fx[i1] > fx[i2]:
            ind[i] = i1
        else:
            ind[i] = i2
    return ind

In [60]:
def mutacao(X, chance):
    [row, col] = X.shape
    Xn = np.zeros((row,col))
    for i in range(row):
        #sorteia um individuo e verifica se ele deve ser mutado
        i1 = random.randint(0,row-1)
        
        if random.random() <= chance:
            #print(i1)
            if random.random()>0.5:
                Xn[i,:] = X[i,:]*1.05
            else:
                Xn[i,:] = X[i,:]*-1.05 
        else:
            Xn[i,:] = X[i,:]
    return Xn

In [61]:
def cruzamento1(X, chance):
    r = X.shape[0]
    c = X.shape[1]
    Xfilho = np.zeros((r,c))
    filho = 0
    alpha = 0.9
    while(filho < 20):
        if random.random() <= chance:
            i1 = random.randint(0,r-1)
            i2 = random.randint(0,r-1)
            Xfilho[filho,:] = alpha*X[i1,:] + (1-alpha)*X[i2,:]
            filho = filho + 1
            Xfilho[filho,:] = alpha*X[i2,:] + (1-alpha)*X[i1,:]
            filho = filho + 1

    return Xfilho

def cruzamento2(X, chance):
    r = X.shape[0]
    c = X.shape[1]
    Xfilho = np.zeros((r,c))
    filho = 0
    alpha = 0.9
    for i in range(int(r/2)):
        i1 = random.randint(0,r-1)
        i2 = random.randint(0,r-1)
        Xfilho[filho,:] = alpha*X[i1,:] + (1-alpha)*X[i2,:]
        filho = filho + 1
        Xfilho[filho,:] = alpha*X[i2,:] + (1-alpha)*X[i1,:]
        filho = filho + 1
    
    return Xfilho

def cruzamento3(X, chance):
    r = X.shape[0]
    c = X.shape[1]
    Xfilho = np.zeros((r,c))
    filho = 0
    for i in range(int(r/2)):
        alpha = round(random.random())
        i1 = random.randint(0,r-1)
        i2 = random.randint(0,r-1)
        Xfilho[filho,:] = alpha*X[i1,:] + (1-alpha)*X[i2,:]
        filho = filho + 1
        Xfilho[filho,:] = alpha*X[i2,:] + (1-alpha)*X[i1,:]
        filho = filho + 1
    
    return Xfilho

In [62]:
def update(X, z, Xn, fxn, taxa):
    auxIndX = np.argsort(z) #indices da ordenação de z
    auxIndXn = np.argsort(fxn) #indices da ordenação de fxn
    
    alterar = round(taxa*num_pop) #quantidade de individuos alterados para a proxima geração.
    
    #substitui os piores da geração atual, pelos melhores da geração anterior
    for i in range(alterar):
        Xn[auxIndXn[i].astype(int)] = X[auxIndX[i].astype(int)]
        
    return Xn

In [63]:
def algoritimoGenetico(cruzamento):
    #armazena o melhor elemento de cada geração
    melhoresIndividuos = np.zeros((30,2))
    melhoresFx = np.zeros((30,2))

    for exect in range(30):
        print("\n----------%dº Execução----------\n" %(exect + 1))
        #Passo 1: geração da população inicial
        X = gera_pop_inicial(num_pop,num_dim, xmin, xmax)

        #Passo 7: Repetição
        for i in range(num_ger):
            #agora devemos avaliar todos os individuos da população inicial
            z = rastrigin(X, A=10)
            fx = inverte(z)

            #descomentar as duas linhas abaixo para mostrar o melhor a cada geração
            melhorind =  np.argmin(z)
            melhorfx = fx[melhorind]

            print("%dº geração: " %(i+1))
            print("Melhor individuo:", end=' ')
            print(X[melhorind])
            print("Fx para esse individuo:", end=' ')
            print(melhorfx, end='\n')

            #Passo 2: realização do torneio
            ind = torneio(fx)
            Xt = popTorneio(ind, X)

            #Passo 3:Cruzamento após o torneio
            if cruzamento == 1 :
                Xf =  cruzamento1(Xt, chancec)
            elif cruzamento == 2 :
                Xf =  cruzamento2(Xt, chancec)
            elif cruzamento == 3 :
                Xf =  cruzamento3(Xt, chancec)

            #Passo 4: Mutação
            Xn = mutacao(Xf, 0.1) 

            zn = rastrigin(Xn, A=10)
            fxn = inverte(zn)

            #Passo 5: Atualização
            taxa = 0.1 #10%
            X = update(X, z, Xn, fxn, taxa)

        #armazena o melhor da g-ésima execução após 100 gerações. Nesse caso, será da função, ao inves da inversão, 
        #pois a inversão depende do mínimo este pode variar a cada iteração 
        z = rastrigin(X, A=10)
        fx = inverte(z)
        melhorind =  np.argmin(z)
        melhoresIndividuos[exect] = X[melhorind]
        melhorfx = fx[melhorind]
        melhoresFx[exect] = melhorfx
        
    return melhoresIndividuos, melhoresFx

<h1>Execução do Algoritmo</h1>

<p>Para esse trabalho serão realizados testes para 3 funções de cruzamento:</p>
<ol>
    <li>Atribuição de chance para geração de filhos após o sorteio dos possíveis pais. A participação é previamente definida para cada pai.</li>
    <li>O sorteio dos pais irá produzir dois filhos. A participação é previamente definida para cada pai.</li>
    <li>A participação de cada pai nos filhos é definida por sorteio.</li>
</ol>

<p>Cada teste, executa 30 vezes o algoritmo genético utilizando uma das funções de cruzamento. Cada execução exibe o melhor indivíduo de cada geração e seu melhor fitness, além de retornar o melhor indivíduo após 100 gerações.</p>

<h2> Teste: cruzamento1 </h2>

<h3>Melhores de cada geração</h3>

In [92]:
bestsIwithC1, bestsFxwithC1 = algoritimoGenetico(1)


----------1º Execução----------

1º geração: 
Melhor individuo: [-0.18580639  0.95169082]
Fx para esse individuo: 1.5848931924611134
2º geração: 
Melhor individuo: [-0.01515574 -0.83834549]
Fx para esse individuo: 1.5848931924611134
3º geração: 
Melhor individuo: [-0.01515574 -0.83834549]
Fx para esse individuo: 1.5848931924611134
4º geração: 
Melhor individuo: [-0.01515574 -0.83834549]
Fx para esse individuo: 1.5848931924611134
5º geração: 
Melhor individuo: [-0.01515574 -0.83834549]
Fx para esse individuo: 1.5848931924611134
6º geração: 
Melhor individuo: [-0.01515574 -0.83834549]
Fx para esse individuo: 1.5848931924611134
7º geração: 
Melhor individuo: [ 0.11536477 -0.0663618 ]
Fx para esse individuo: 1.5848931924611134
8º geração: 
Melhor individuo: [ 0.10462315 -0.03823511]
Fx para esse individuo: 1.5848931924611134
9º geração: 
Melhor individuo: [0.0902624  0.01949463]
Fx para esse individuo: 1.5848931924611134
10º geração: 
Melhor individuo: [-0.0701342   0.01829168]
Fx para es

92º geração: 
Melhor individuo: [ 0.0034926  -0.00351551]
Fx para esse individuo: 1.5848931924611134
93º geração: 
Melhor individuo: [ 0.0034926  -0.00351551]
Fx para esse individuo: 1.5848931924611134
94º geração: 
Melhor individuo: [ 0.0034926  -0.00351551]
Fx para esse individuo: 1.5848931924611134
95º geração: 
Melhor individuo: [ 0.0034926  -0.00351551]
Fx para esse individuo: 1.5848931924611134
96º geração: 
Melhor individuo: [ 0.0034926  -0.00351551]
Fx para esse individuo: 1.5848931924611134
97º geração: 
Melhor individuo: [ 0.0034926  -0.00351551]
Fx para esse individuo: 1.5848931924611134
98º geração: 
Melhor individuo: [ 0.0034926  -0.00351551]
Fx para esse individuo: 1.5848931924611134
99º geração: 
Melhor individuo: [ 0.0034926  -0.00351551]
Fx para esse individuo: 1.5848931924611134
100º geração: 
Melhor individuo: [ 0.0034926  -0.00351551]
Fx para esse individuo: 1.5848931924611134

----------2º Execução----------

1º geração: 
Melhor individuo: [1.03860007 1.76582142]
F

31º geração: 
Melhor individuo: [-2.01753423 -0.98363755]
Fx para esse individuo: 1.5848931924611134
32º geração: 
Melhor individuo: [-2.01753423 -0.98363755]
Fx para esse individuo: 1.5848931924611134
33º geração: 
Melhor individuo: [-2.01753423 -0.98363755]
Fx para esse individuo: 1.5848931924611134
34º geração: 
Melhor individuo: [-2.01753423 -0.98363755]
Fx para esse individuo: 1.5848931924611134
35º geração: 
Melhor individuo: [-2.01753423 -0.98363755]
Fx para esse individuo: 1.5848931924611134
36º geração: 
Melhor individuo: [-2.01753423 -0.98363755]
Fx para esse individuo: 1.5848931924611134
37º geração: 
Melhor individuo: [-2.01753423 -0.98363755]
Fx para esse individuo: 1.5848931924611134
38º geração: 
Melhor individuo: [-2.01753423 -0.98363755]
Fx para esse individuo: 1.5848931924611134
39º geração: 
Melhor individuo: [-2.01753423 -0.98363755]
Fx para esse individuo: 1.5848931924611134
40º geração: 
Melhor individuo: [-2.01753423 -0.98363755]
Fx para esse individuo: 1.5848931

Fx para esse individuo: 1.5848931924611134
31º geração: 
Melhor individuo: [-1.00472843  0.00744738]
Fx para esse individuo: 1.5848931924611134
32º geração: 
Melhor individuo: [-1.00472843  0.00744738]
Fx para esse individuo: 1.5848931924611134
33º geração: 
Melhor individuo: [-1.00472843  0.00744738]
Fx para esse individuo: 1.5848931924611134
34º geração: 
Melhor individuo: [-1.00472843  0.00744738]
Fx para esse individuo: 1.5848931924611134
35º geração: 
Melhor individuo: [-1.00472843  0.00744738]
Fx para esse individuo: 1.5848931924611134
36º geração: 
Melhor individuo: [-1.00472843  0.00744738]
Fx para esse individuo: 1.5848931924611134
37º geração: 
Melhor individuo: [-1.00472843  0.00744738]
Fx para esse individuo: 1.5848931924611134
38º geração: 
Melhor individuo: [-1.00472843  0.00744738]
Fx para esse individuo: 1.5848931924611134
39º geração: 
Melhor individuo: [-1.00472843  0.00744738]
Fx para esse individuo: 1.5848931924611134
40º geração: 
Melhor individuo: [-1.00472843  0.

Melhor individuo: [0.12956479 1.09852265]
Fx para esse individuo: 1.5848931924611134
31º geração: 
Melhor individuo: [0.12956479 1.09852265]
Fx para esse individuo: 1.5848931924611134
32º geração: 
Melhor individuo: [0.12956479 1.09852265]
Fx para esse individuo: 1.5848931924611134
33º geração: 
Melhor individuo: [0.12956479 1.09852265]
Fx para esse individuo: 1.5848931924611134
34º geração: 
Melhor individuo: [0.12956479 1.09852265]
Fx para esse individuo: 1.5848931924611134
35º geração: 
Melhor individuo: [0.12956479 1.09852265]
Fx para esse individuo: 1.5848931924611134
36º geração: 
Melhor individuo: [0.12956479 1.09852265]
Fx para esse individuo: 1.5848931924611134
37º geração: 
Melhor individuo: [0.12956479 1.09852265]
Fx para esse individuo: 1.5848931924611134
38º geração: 
Melhor individuo: [0.12956479 1.09852265]
Fx para esse individuo: 1.5848931924611134
39º geração: 
Melhor individuo: [0.12956479 1.09852265]
Fx para esse individuo: 1.5848931924611134
40º geração: 
Melhor ind

...

24º geração: 
Melhor individuo: [-0.97438368  0.06195279]
Fx para esse individuo: 1.5848931924611134
25º geração: 
Melhor individuo: [-0.97438368  0.06195279]
Fx para esse individuo: 1.5848931924611134
26º geração: 
Melhor individuo: [-0.97438368  0.06195279]
Fx para esse individuo: 1.5848931924611134
27º geração: 
Melhor individuo: [-0.97438368  0.06195279]
Fx para esse individuo: 1.5848931924611134
28º geração: 
Melhor individuo: [-0.97438368  0.06195279]
Fx para esse individuo: 1.5848931924611134
29º geração: 
Melhor individuo: [-0.97438368  0.06195279]
Fx para esse individuo: 1.5848931924611134
30º geração: 
Melhor individuo: [-0.9792556   0.06226256]
Fx para esse individuo: 1.5848931924611134
31º geração: 
Melhor individuo: [-0.9792556   0.06226256]
Fx para esse individuo: 1.5848931924611134
32º geração: 
Melhor individuo: [-0.9792556   0.06226256]
Fx para esse individuo: 1.5848931924611134
33º geração: 
Melhor individuo: [-0.9792556   0.06226256]
Fx para esse individuo: 1.5848931

Fx para esse individuo: 1.5848931924611134
24º geração: 
Melhor individuo: [-0.02503749  0.01339289]
Fx para esse individuo: 1.5848931924611134
25º geração: 
Melhor individuo: [ 0.01270131 -0.02224038]
Fx para esse individuo: 1.5848931924611134
26º geração: 
Melhor individuo: [ 0.00887428 -0.01862686]
Fx para esse individuo: 1.5848931924611134
27º geração: 
Melhor individuo: [-0.01835443  0.0064976 ]
Fx para esse individuo: 1.5848931924611134
28º geração: 
Melhor individuo: [-0.01559329  0.00394902]
Fx para esse individuo: 1.5848931924611134
29º geração: 
Melhor individuo: [-0.01341347  0.00194347]
Fx para esse individuo: 1.5848931924611134
30º geração: 
Melhor individuo: [-0.01170097  0.00036852]
Fx para esse individuo: 1.5848931924611134
31º geração: 
Melhor individuo: [-0.01011823 -0.00113905]
Fx para esse individuo: 1.5848931924611134
32º geração: 
Melhor individuo: [-0.00887863 -0.00227447]
Fx para esse individuo: 1.5848931924611134
33º geração: 
Melhor individuo: [-0.00887863 -0.

Melhor individuo: [0.03411649 0.99304712]
Fx para esse individuo: 1.5848931924611134
24º geração: 
Melhor individuo: [0.03411649 0.99304712]
Fx para esse individuo: 1.5848931924611134
25º geração: 
Melhor individuo: [0.03411649 0.99304712]
Fx para esse individuo: 1.5848931924611134
26º geração: 
Melhor individuo: [0.03411649 0.99304712]
Fx para esse individuo: 1.5848931924611134
27º geração: 
Melhor individuo: [0.03411649 0.99304712]
Fx para esse individuo: 1.5848931924611134
28º geração: 
Melhor individuo: [0.03411649 0.99304712]
Fx para esse individuo: 1.5848931924611134
29º geração: 
Melhor individuo: [0.03411649 0.99304712]
Fx para esse individuo: 1.5848931924611134
30º geração: 
Melhor individuo: [0.03411649 0.99304712]
Fx para esse individuo: 1.5848931924611134
31º geração: 
Melhor individuo: [0.03411649 0.99304712]
Fx para esse individuo: 1.5848931924611134
32º geração: 
Melhor individuo: [0.03411649 0.99304712]
Fx para esse individuo: 1.5848931924611134
33º geração: 
Melhor ind

Melhor individuo: [-0.02602816 -0.99393852]
Fx para esse individuo: 1.5848931924611134
24º geração: 
Melhor individuo: [-0.02602816 -0.99393852]
Fx para esse individuo: 1.5848931924611134
25º geração: 
Melhor individuo: [-0.02603028 -0.99401994]
Fx para esse individuo: 1.5848931924611134
26º geração: 
Melhor individuo: [-0.02603028 -0.99401994]
Fx para esse individuo: 1.5848931924611134
27º geração: 
Melhor individuo: [-0.02603028 -0.99401994]
Fx para esse individuo: 1.5848931924611134
28º geração: 
Melhor individuo: [-0.02603028 -0.99401994]
Fx para esse individuo: 1.5848931924611134
29º geração: 
Melhor individuo: [-0.02603028 -0.99401994]
Fx para esse individuo: 1.5848931924611134
30º geração: 
Melhor individuo: [-0.02603028 -0.99401994]
Fx para esse individuo: 1.5848931924611134
31º geração: 
Melhor individuo: [-0.02603028 -0.99401994]
Fx para esse individuo: 1.5848931924611134
32º geração: 
Melhor individuo: [-0.02603028 -0.99401994]
Fx para esse individuo: 1.5848931924611134
33º 

Fx para esse individuo: 1.5848931924611134
23º geração: 
Melhor individuo: [1.02407907 0.95240233]
Fx para esse individuo: 1.5848931924611134
24º geração: 
Melhor individuo: [1.02407907 0.95240233]
Fx para esse individuo: 1.5848931924611134
25º geração: 
Melhor individuo: [1.02407907 0.95240233]
Fx para esse individuo: 1.5848931924611134
26º geração: 
Melhor individuo: [1.02407907 0.95240233]
Fx para esse individuo: 1.5848931924611134
27º geração: 
Melhor individuo: [1.02407907 0.95240233]
Fx para esse individuo: 1.5848931924611134
28º geração: 
Melhor individuo: [1.02407907 0.95240233]
Fx para esse individuo: 1.5848931924611134
29º geração: 
Melhor individuo: [1.02407907 0.95240233]
Fx para esse individuo: 1.5848931924611134
30º geração: 
Melhor individuo: [1.02407907 0.95240233]
Fx para esse individuo: 1.5848931924611134
31º geração: 
Melhor individuo: [1.02407907 0.95240233]
Fx para esse individuo: 1.5848931924611134
32º geração: 
Melhor individuo: [1.02407907 0.95240233]
Fx para es

<h3>Melhores Indivíduos de cada Execução</h3>

In [93]:
bestsIwithC1

array([[ 3.49260047e-03, -3.51551238e-03],
       [ 9.91689460e-01, -2.84298258e-02],
       [-2.01753423e+00, -9.83637547e-01],
       [-1.00472843e+00,  7.44738104e-03],
       [ 1.29564785e-01,  1.09852265e+00],
       [ 9.62570272e-01,  1.02750028e+00],
       [-1.97392578e+00,  2.95360942e-02],
       [-1.04926127e+00, -9.64047182e-01],
       [-4.20010343e-02,  9.94952150e-01],
       [-1.01306429e+00,  9.84973054e-01],
       [ 1.63397724e-02,  9.97743929e-01],
       [-1.12354581e+00,  9.73173099e-01],
       [ 1.01629435e+00, -6.32233464e-02],
       [ 2.04581430e-02, -9.94052398e-01],
       [ 9.95808283e-01,  9.72970731e-01],
       [-1.63823390e-02, -9.94969535e-01],
       [ 9.89406454e-01, -1.99266131e+00],
       [ 9.81881833e-01,  1.99821593e+00],
       [-1.98338964e+00, -1.41056610e-01],
       [ 7.00704020e-03,  6.48890941e-03],
       [ 6.06346782e-04,  9.97341580e-01],
       [-2.05951399e+00, -3.37644736e-02],
       [ 3.01050103e-05,  1.03212985e-04],
       [-9.

<h3>Melhores Fitness de cada Execução</h3>


In [94]:
bestsFxwithC1

array([[1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319]])

<h2> Teste: cruzamento2 </h2>

<h3>Melhores de cada geração</h3>

In [95]:
bestsIwithC2, bestsFxwithC2 = algoritimoGenetico(2)


----------1º Execução----------

1º geração: 
Melhor individuo: [-0.01873914  3.05196989]
Fx para esse individuo: 1.5848931924611112
2º geração: 
Melhor individuo: [-0.06480252  1.92297134]
Fx para esse individuo: 1.5848931924611134
3º geração: 
Melhor individuo: [ 0.01390956 -0.86487645]
Fx para esse individuo: 1.5848931924611134
4º geração: 
Melhor individuo: [ 0.01386221 -0.9911295 ]
Fx para esse individuo: 1.5848931924611134
5º geração: 
Melhor individuo: [ 0.01386221 -0.9911295 ]
Fx para esse individuo: 1.5848931924611134
6º geração: 
Melhor individuo: [ 0.01405996 -0.99363825]
Fx para esse individuo: 1.5848931924611134
7º geração: 
Melhor individuo: [ 0.01407148 -0.99532902]
Fx para esse individuo: 1.5848931924611134
8º geração: 
Melhor individuo: [ 0.01407066 -0.9949245 ]
Fx para esse individuo: 1.5848931924611134
9º geração: 
Melhor individuo: [ 0.01405253 -0.99493415]
Fx para esse individuo: 1.5848931924611134
10º geração: 
Melhor individuo: [ 0.01405253 -0.99493415]
Fx para 

18º geração: 
Melhor individuo: [-0.0027223  -0.00488455]
Fx para esse individuo: 1.5848931924611134
19º geração: 
Melhor individuo: [-0.00273077 -0.00378254]
Fx para esse individuo: 1.5848931924611134
20º geração: 
Melhor individuo: [0.00230901 0.0031012 ]
Fx para esse individuo: 1.5848931924611134
21º geração: 
Melhor individuo: [0.00230901 0.0031012 ]
Fx para esse individuo: 1.5848931924611134
22º geração: 
Melhor individuo: [0.0018561  0.00239367]
Fx para esse individuo: 1.5848931924611134
23º geração: 
Melhor individuo: [0.00139116 0.00239131]
Fx para esse individuo: 1.5848931924611134
24º geração: 
Melhor individuo: [0.00101285 0.00193492]
Fx para esse individuo: 1.5848931924611134
25º geração: 
Melhor individuo: [-0.00160119 -0.00129308]
Fx para esse individuo: 1.5848931924611134
26º geração: 
Melhor individuo: [-0.00134328 -0.0009305 ]
Fx para esse individuo: 1.5848931924611134
27º geração: 
Melhor individuo: [-0.00107444 -0.00063623]
Fx para esse individuo: 1.5848931924611134


Fx para esse individuo: 1.5848931924611158
18º geração: 
Melhor individuo: [0.01468203 2.99615165]
Fx para esse individuo: 1.5848931924611158
19º geração: 
Melhor individuo: [0.01468203 2.99615165]
Fx para esse individuo: 1.5848931924611158
20º geração: 
Melhor individuo: [0.01468203 2.99615165]
Fx para esse individuo: 1.5848931924611158
21º geração: 
Melhor individuo: [0.01468203 2.99615165]
Fx para esse individuo: 1.5848931924611158
22º geração: 
Melhor individuo: [0.01468203 2.99615165]
Fx para esse individuo: 1.5848931924611158
23º geração: 
Melhor individuo: [0.01468203 2.99615165]
Fx para esse individuo: 1.5848931924611158
24º geração: 
Melhor individuo: [0.01468203 2.99615165]
Fx para esse individuo: 1.5848931924611158
25º geração: 
Melhor individuo: [0.01468203 2.99615165]
Fx para esse individuo: 1.5848931924611158
26º geração: 
Melhor individuo: [0.01468203 2.99615165]
Fx para esse individuo: 1.5848931924611158
27º geração: 
Melhor individuo: [0.01468203 2.99615165]
Fx para es

Melhor individuo: [-1.95951763  1.02396944]
Fx para esse individuo: 1.5848931924611134
18º geração: 
Melhor individuo: [-1.95951763  1.02396944]
Fx para esse individuo: 1.5848931924611134
19º geração: 
Melhor individuo: [-1.95951763  1.02396944]
Fx para esse individuo: 1.5848931924611134
20º geração: 
Melhor individuo: [-1.95951763  1.02396944]
Fx para esse individuo: 1.5848931924611134
21º geração: 
Melhor individuo: [-1.95951763  1.02396944]
Fx para esse individuo: 1.5848931924611134
22º geração: 
Melhor individuo: [-1.95951763  1.02396944]
Fx para esse individuo: 1.5848931924611134
23º geração: 
Melhor individuo: [-1.95951763  1.02396944]
Fx para esse individuo: 1.5848931924611134
24º geração: 
Melhor individuo: [-1.95951763  1.02396944]
Fx para esse individuo: 1.5848931924611134
25º geração: 
Melhor individuo: [-1.95951763  1.02396944]
Fx para esse individuo: 1.5848931924611134
26º geração: 
Melhor individuo: [-1.95951763  1.02396944]
Fx para esse individuo: 1.5848931924611134
27º 

17º geração: 
Melhor individuo: [0.98017927 0.00258527]
Fx para esse individuo: 1.5848931924611134
18º geração: 
Melhor individuo: [0.98017927 0.00258527]
Fx para esse individuo: 1.5848931924611134
19º geração: 
Melhor individuo: [0.98017927 0.00258527]
Fx para esse individuo: 1.5848931924611134
20º geração: 
Melhor individuo: [0.98017927 0.00258527]
Fx para esse individuo: 1.5848931924611134
21º geração: 
Melhor individuo: [0.98017927 0.00258527]
Fx para esse individuo: 1.5848931924611134
22º geração: 
Melhor individuo: [0.98017927 0.00258527]
Fx para esse individuo: 1.5848931924611134
23º geração: 
Melhor individuo: [0.98017927 0.00258527]
Fx para esse individuo: 1.5848931924611134
24º geração: 
Melhor individuo: [0.98017927 0.00258527]
Fx para esse individuo: 1.5848931924611134
25º geração: 
Melhor individuo: [0.98508016 0.0025982 ]
Fx para esse individuo: 1.5848931924611134
26º geração: 
Melhor individuo: [0.98508016 0.0025982 ]
Fx para esse individuo: 1.5848931924611134
27º geraçã

...

Fx para esse individuo: 1.5848931924611134
11º geração: 
Melhor individuo: [-0.95800729 -0.04490654]
Fx para esse individuo: 1.5848931924611134
12º geração: 
Melhor individuo: [-0.98576366 -0.03623645]
Fx para esse individuo: 1.5848931924611134
13º geração: 
Melhor individuo: [-0.98576366 -0.03623645]
Fx para esse individuo: 1.5848931924611134
14º geração: 
Melhor individuo: [-0.98576366 -0.03623645]
Fx para esse individuo: 1.5848931924611134
15º geração: 
Melhor individuo: [-0.98576366 -0.03623645]
Fx para esse individuo: 1.5848931924611134
16º geração: 
Melhor individuo: [-0.98576366 -0.03623645]
Fx para esse individuo: 1.5848931924611134
17º geração: 
Melhor individuo: [-0.98576366 -0.03623645]
Fx para esse individuo: 1.5848931924611134
18º geração: 
Melhor individuo: [-0.98576366 -0.03623645]
Fx para esse individuo: 1.5848931924611134
19º geração: 
Melhor individuo: [-0.98576366 -0.03623645]
Fx para esse individuo: 1.5848931924611134
20º geração: 
Melhor individuo: [-0.98576366 -0.

Melhor individuo: [-0.96923513 -0.02304456]
Fx para esse individuo: 1.5848931924611134
11º geração: 
Melhor individuo: [-0.97555983 -0.02078381]
Fx para esse individuo: 1.5848931924611134
12º geração: 
Melhor individuo: [-0.97555983 -0.02078381]
Fx para esse individuo: 1.5848931924611134
13º geração: 
Melhor individuo: [-0.98061959 -0.01897521]
Fx para esse individuo: 1.5848931924611134
14º geração: 
Melhor individuo: [-0.9846838  -0.01754667]
Fx para esse individuo: 1.5848931924611134
15º geração: 
Melhor individuo: [-0.98958366 -0.01756199]
Fx para esse individuo: 1.5848931924611134
16º geração: 
Melhor individuo: [-0.99175531 -0.01635159]
Fx para esse individuo: 1.5848931924611134
17º geração: 
Melhor individuo: [-0.99175531 -0.01635159]
Fx para esse individuo: 1.5848931924611134
18º geração: 
Melhor individuo: [-0.99175531 -0.01635159]
Fx para esse individuo: 1.5848931924611134
19º geração: 
Melhor individuo: [-0.99175531 -0.01635159]
Fx para esse individuo: 1.5848931924611134
20º 

Melhor individuo: [-0.03056122 -1.02072965]
Fx para esse individuo: 1.5848931924611134
11º geração: 
Melhor individuo: [-0.03056122 -1.02072965]
Fx para esse individuo: 1.5848931924611134
12º geração: 
Melhor individuo: [-0.03056122 -1.02072965]
Fx para esse individuo: 1.5848931924611134
13º geração: 
Melhor individuo: [-0.03056122 -1.02072965]
Fx para esse individuo: 1.5848931924611134
14º geração: 
Melhor individuo: [-0.03056122 -1.02072965]
Fx para esse individuo: 1.5848931924611134
15º geração: 
Melhor individuo: [-0.03056122 -1.02072965]
Fx para esse individuo: 1.5848931924611134
16º geração: 
Melhor individuo: [-0.03056122 -1.02072965]
Fx para esse individuo: 1.5848931924611134
17º geração: 
Melhor individuo: [-0.03056122 -1.02072965]
Fx para esse individuo: 1.5848931924611134
18º geração: 
Melhor individuo: [-0.03056122 -1.02072965]
Fx para esse individuo: 1.5848931924611134
19º geração: 
Melhor individuo: [-0.03056122 -1.02072965]
Fx para esse individuo: 1.5848931924611134
20º 

Fx para esse individuo: 1.5848931924611134
10º geração: 
Melhor individuo: [-0.98410424  0.99357789]
Fx para esse individuo: 1.5848931924611134
11º geração: 
Melhor individuo: [-0.98410424  0.99357789]
Fx para esse individuo: 1.5848931924611134
12º geração: 
Melhor individuo: [-0.98449182  0.99467037]
Fx para esse individuo: 1.5848931924611134
13º geração: 
Melhor individuo: [-0.98480543  0.99562515]
Fx para esse individuo: 1.5848931924611134
14º geração: 
Melhor individuo: [-0.98504549  0.99636705]
Fx para esse individuo: 1.5848931924611134
15º geração: 
Melhor individuo: [-0.98524414  0.99697075]
Fx para esse individuo: 1.5848931924611134
16º geração: 
Melhor individuo: [-0.98544833  0.99759142]
Fx para esse individuo: 1.5848931924611134
17º geração: 
Melhor individuo: [-0.98544833  0.99759142]
Fx para esse individuo: 1.5848931924611134
18º geração: 
Melhor individuo: [-0.98544833  0.99759142]
Fx para esse individuo: 1.5848931924611134
19º geração: 
Melhor individuo: [-0.98544833  0.

Fx para esse individuo: 1.5848931924611134
10º geração: 
Melhor individuo: [-2.01803314  0.9163785 ]
Fx para esse individuo: 1.5848931924611134
11º geração: 
Melhor individuo: [-2.01803314  0.9163785 ]
Fx para esse individuo: 1.5848931924611134
12º geração: 
Melhor individuo: [-2.01803314  0.9163785 ]
Fx para esse individuo: 1.5848931924611134
13º geração: 
Melhor individuo: [-2.01803314  0.9163785 ]
Fx para esse individuo: 1.5848931924611134
14º geração: 
Melhor individuo: [-2.01803314  0.9163785 ]
Fx para esse individuo: 1.5848931924611134
15º geração: 
Melhor individuo: [-2.01803314  0.9163785 ]
Fx para esse individuo: 1.5848931924611134
16º geração: 
Melhor individuo: [-2.01803314  0.9163785 ]
Fx para esse individuo: 1.5848931924611134
17º geração: 
Melhor individuo: [-2.01803314  0.9163785 ]
Fx para esse individuo: 1.5848931924611134
18º geração: 
Melhor individuo: [-2.01803314  0.9163785 ]
Fx para esse individuo: 1.5848931924611134
19º geração: 
Melhor individuo: [-2.01803314  0.

<h3>Melhores Indivíduos de cada Execução</h3>

In [96]:
bestsIwithC2

array([[ 1.40525324e-02, -9.94934153e-01],
       [ 1.99498224e-04,  5.23412045e-05],
       [ 1.46820316e-02,  2.99615165e+00],
       [-1.95951763e+00,  1.02396944e+00],
       [ 9.90005561e-01,  2.61118637e-03],
       [ 9.40522358e-03, -1.01972087e+00],
       [ 9.48077014e-01, -1.05096559e+00],
       [ 1.99822286e+00, -2.22027592e-02],
       [ 1.04558842e-03, -9.94969795e-01],
       [ 1.01142945e+00,  1.03963420e+00],
       [ 4.58811845e-03,  1.66986222e-02],
       [ 9.76314429e-01,  1.01350781e+00],
       [-1.98963076e+00,  1.95691990e-02],
       [-9.79365385e-01,  1.00644982e+00],
       [ 1.97230916e+00, -1.02831228e+00],
       [ 1.60488505e-02,  2.10280010e+00],
       [-5.86521646e-03, -1.00044847e+00],
       [ 1.01241586e+00, -3.60065223e-02],
       [ 6.30556792e-06, -9.01742931e-06],
       [-9.62355995e-01, -1.98905393e+00],
       [ 5.82725239e-03,  1.02929415e+00],
       [-9.91957013e-01, -4.79375691e-02],
       [-1.98238162e+00, -1.99308460e+00],
       [-1.

<h3>Melhores Fitness de cada Execução</h3>

In [97]:
bestsFxwithC2

array([[1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319]])

<h2> Teste: cruzamento3 </h2>

<h3>Melhores de cada geração</h3>

In [98]:
bestsIwithC3, bestsFxwithC3 = algoritimoGenetico(3)


----------1º Execução----------

1º geração: 
Melhor individuo: [ 0.17680811 -3.05646086]
Fx para esse individuo: 1.5848931924611112
2º geração: 
Melhor individuo: [ 0.17680811 -3.05646086]
Fx para esse individuo: 1.5848931924611112
3º geração: 
Melhor individuo: [ 0.17680811 -3.05646086]
Fx para esse individuo: 1.5848931924611112
4º geração: 
Melhor individuo: [ 0.17680811 -3.05646086]
Fx para esse individuo: 1.5848931924611112
5º geração: 
Melhor individuo: [ 0.17680811 -3.05646086]
Fx para esse individuo: 1.5848931924611112
6º geração: 
Melhor individuo: [ 0.17680811 -3.05646086]
Fx para esse individuo: 1.5848931924611112
7º geração: 
Melhor individuo: [ 0.17680811 -3.05646086]
Fx para esse individuo: 1.5848931924611112
8º geração: 
Melhor individuo: [ 0.17680811 -3.05646086]
Fx para esse individuo: 1.5848931924611112
9º geração: 
Melhor individuo: [ 0.17680811 -3.05646086]
Fx para esse individuo: 1.5848931924611112
10º geração: 
Melhor individuo: [ 0.17680811 -3.05646086]
Fx para 

80º geração: 
Melhor individuo: [ 0.00892091 -3.93018744]
Fx para esse individuo: 1.5848931924611112
81º geração: 
Melhor individuo: [ 0.00892091 -3.93018744]
Fx para esse individuo: 1.5848931924611112
82º geração: 
Melhor individuo: [ 0.00892091 -3.93018744]
Fx para esse individuo: 1.5848931924611112
83º geração: 
Melhor individuo: [ 0.00892091 -3.93018744]
Fx para esse individuo: 1.5848931924611112
84º geração: 
Melhor individuo: [ 0.00892091 -3.93018744]
Fx para esse individuo: 1.5848931924611112
85º geração: 
Melhor individuo: [ 0.00892091 -3.93018744]
Fx para esse individuo: 1.5848931924611112
86º geração: 
Melhor individuo: [ 0.00892091 -3.93018744]
Fx para esse individuo: 1.5848931924611112
87º geração: 
Melhor individuo: [ 0.00892091 -3.93018744]
Fx para esse individuo: 1.5848931924611112
88º geração: 
Melhor individuo: [ 0.00892091 -3.93018744]
Fx para esse individuo: 1.5848931924611112
89º geração: 
Melhor individuo: [ 0.00892091 -3.93018744]
Fx para esse individuo: 1.5848931

Melhor individuo: [-0.0931114   2.18340687]
Fx para esse individuo: 1.5848931924611112
86º geração: 
Melhor individuo: [-0.0931114   2.18340687]
Fx para esse individuo: 1.5848931924611112
87º geração: 
Melhor individuo: [-0.0931114   2.18340687]
Fx para esse individuo: 1.5848931924611112
88º geração: 
Melhor individuo: [-0.0931114   2.18340687]
Fx para esse individuo: 1.5848931924611112
89º geração: 
Melhor individuo: [-0.0931114   2.18340687]
Fx para esse individuo: 1.5848931924611112
90º geração: 
Melhor individuo: [-0.0931114   2.18340687]
Fx para esse individuo: 1.5848931924611112
91º geração: 
Melhor individuo: [-0.0931114   2.18340687]
Fx para esse individuo: 1.5848931924611112
92º geração: 
Melhor individuo: [-0.0931114   2.18340687]
Fx para esse individuo: 1.5848931924611112
93º geração: 
Melhor individuo: [-0.0931114   2.18340687]
Fx para esse individuo: 1.5848931924611112
94º geração: 
Melhor individuo: [-0.0931114   2.18340687]
Fx para esse individuo: 1.5848931924611112
95º 

Fx para esse individuo: 1.5848931924611134
4º geração: 
Melhor individuo: [ 1.05863179 -0.90402091]
Fx para esse individuo: 1.5848931924611134
5º geração: 
Melhor individuo: [ 1.05863179 -0.90402091]
Fx para esse individuo: 1.5848931924611134
6º geração: 
Melhor individuo: [ 1.05863179 -0.90402091]
Fx para esse individuo: 1.5848931924611134
7º geração: 
Melhor individuo: [ 1.05863179 -0.90402091]
Fx para esse individuo: 1.5848931924611134
8º geração: 
Melhor individuo: [ 1.05863179 -0.90402091]
Fx para esse individuo: 1.5848931924611134
9º geração: 
Melhor individuo: [ 1.05863179 -0.90402091]
Fx para esse individuo: 1.5848931924611134
10º geração: 
Melhor individuo: [ 1.05863179 -0.90402091]
Fx para esse individuo: 1.5848931924611134
11º geração: 
Melhor individuo: [ 1.05863179 -0.90402091]
Fx para esse individuo: 1.5848931924611134
12º geração: 
Melhor individuo: [ 1.05863179 -0.90402091]
Fx para esse individuo: 1.5848931924611134
13º geração: 
Melhor individuo: [ 1.05863179 -0.904020

Fx para esse individuo: 1.5848931924611134
4º geração: 
Melhor individuo: [ 0.10828198 -0.08632637]
Fx para esse individuo: 1.5848931924611134
5º geração: 
Melhor individuo: [ 0.10828198 -0.08632637]
Fx para esse individuo: 1.5848931924611134
6º geração: 
Melhor individuo: [ 0.10828198 -0.08632637]
Fx para esse individuo: 1.5848931924611134
7º geração: 
Melhor individuo: [ 0.10828198 -0.08632637]
Fx para esse individuo: 1.5848931924611134
8º geração: 
Melhor individuo: [ 0.10828198 -0.08632637]
Fx para esse individuo: 1.5848931924611134
9º geração: 
Melhor individuo: [ 0.10828198 -0.08632637]
Fx para esse individuo: 1.5848931924611134
10º geração: 
Melhor individuo: [ 0.10828198 -0.08632637]
Fx para esse individuo: 1.5848931924611134
11º geração: 
Melhor individuo: [ 0.10828198 -0.08632637]
Fx para esse individuo: 1.5848931924611134
12º geração: 
Melhor individuo: [ 0.10828198 -0.08632637]
Fx para esse individuo: 1.5848931924611134
13º geração: 
Melhor individuo: [ 0.10828198 -0.086326

...

Melhor individuo: [-0.83068409  1.09537106]
Fx para esse individuo: 1.5848931924611158
98º geração: 
Melhor individuo: [-0.83068409  1.09537106]
Fx para esse individuo: 1.5848931924611158
99º geração: 
Melhor individuo: [-0.83068409  1.09537106]
Fx para esse individuo: 1.5848931924611158
100º geração: 
Melhor individuo: [-0.83068409  1.09537106]
Fx para esse individuo: 1.5848931924611158

----------27º Execução----------

1º geração: 
Melhor individuo: [-0.98998845 -0.72404319]
Fx para esse individuo: 1.5848931924611112
2º geração: 
Melhor individuo: [-0.98998845 -0.72404319]
Fx para esse individuo: 1.5848931924611112
3º geração: 
Melhor individuo: [-0.9494134  -0.78631645]
Fx para esse individuo: 1.5848931924611158
4º geração: 
Melhor individuo: [-0.9494134  -0.78631645]
Fx para esse individuo: 1.5848931924611158
5º geração: 
Melhor individuo: [-0.9494134  -0.78631645]
Fx para esse individuo: 1.5848931924611158
6º geração: 
Melhor individuo: [-0.99688407 -0.82563227]
Fx para esse indi

Melhor individuo: [1.09906469 0.91025958]
Fx para esse individuo: 1.5848931924611134
98º geração: 
Melhor individuo: [1.09906469 0.91025958]
Fx para esse individuo: 1.5848931924611134
99º geração: 
Melhor individuo: [1.09906469 0.91025958]
Fx para esse individuo: 1.5848931924611134
100º geração: 
Melhor individuo: [1.09906469 0.91025958]
Fx para esse individuo: 1.5848931924611134

----------28º Execução----------

1º geração: 
Melhor individuo: [-0.12885632  0.93297204]
Fx para esse individuo: 1.5848931924611134
2º geração: 
Melhor individuo: [-0.12885632  0.93297204]
Fx para esse individuo: 1.5848931924611134
3º geração: 
Melhor individuo: [-0.13529914  0.97962064]
Fx para esse individuo: 1.5848931924611134
4º geração: 
Melhor individuo: [-0.13529914  0.97962064]
Fx para esse individuo: 1.5848931924611134
5º geração: 
Melhor individuo: [-0.13529914  0.97962064]
Fx para esse individuo: 1.5848931924611134
6º geração: 
Melhor individuo: [-0.13529914  0.97962064]
Fx para esse individuo: 1

Fx para esse individuo: 1.5848931924611134
97º geração: 
Melhor individuo: [-0.13529914  0.97962064]
Fx para esse individuo: 1.5848931924611134
98º geração: 
Melhor individuo: [-0.13529914  0.97962064]
Fx para esse individuo: 1.5848931924611134
99º geração: 
Melhor individuo: [-0.13529914  0.97962064]
Fx para esse individuo: 1.5848931924611134
100º geração: 
Melhor individuo: [-0.13529914  0.97962064]
Fx para esse individuo: 1.5848931924611134

----------29º Execução----------

1º geração: 
Melhor individuo: [ 2.05230518 -1.0185242 ]
Fx para esse individuo: 1.5848931924611134
2º geração: 
Melhor individuo: [ 2.05230518 -1.0185242 ]
Fx para esse individuo: 1.5848931924611134
3º geração: 
Melhor individuo: [ 2.05230518 -1.0185242 ]
Fx para esse individuo: 1.5848931924611134
4º geração: 
Melhor individuo: [ 2.05230518 -1.0185242 ]
Fx para esse individuo: 1.5848931924611134
5º geração: 
Melhor individuo: [ 2.05230518 -1.0185242 ]
Fx para esse individuo: 1.5848931924611134
6º geração: 
Melh

Fx para esse individuo: 1.5848931924611134
97º geração: 
Melhor individuo: [ 2.05230518 -1.0185242 ]
Fx para esse individuo: 1.5848931924611134
98º geração: 
Melhor individuo: [ 2.05230518 -1.0185242 ]
Fx para esse individuo: 1.5848931924611134
99º geração: 
Melhor individuo: [ 2.05230518 -1.0185242 ]
Fx para esse individuo: 1.5848931924611134
100º geração: 
Melhor individuo: [ 2.05230518 -1.0185242 ]
Fx para esse individuo: 1.5848931924611134

----------30º Execução----------

1º geração: 
Melhor individuo: [-0.948261   -1.88289343]
Fx para esse individuo: 1.5848931924611134
2º geração: 
Melhor individuo: [-0.948261   -1.88289343]
Fx para esse individuo: 1.5848931924611134
3º geração: 
Melhor individuo: [-0.948261   -1.88289343]
Fx para esse individuo: 1.5848931924611134
4º geração: 
Melhor individuo: [0.99567405 1.9770381 ]
Fx para esse individuo: 1.5848931924611134
5º geração: 
Melhor individuo: [0.99567405 1.9770381 ]
Fx para esse individuo: 1.5848931924611134
6º geração: 
Melhor i

Melhor individuo: [0.99567405 1.9770381 ]
Fx para esse individuo: 1.5848931924611134
97º geração: 
Melhor individuo: [0.99567405 1.9770381 ]
Fx para esse individuo: 1.5848931924611134
98º geração: 
Melhor individuo: [0.99567405 1.9770381 ]
Fx para esse individuo: 1.5848931924611134
99º geração: 
Melhor individuo: [0.99567405 1.9770381 ]
Fx para esse individuo: 1.5848931924611134
100º geração: 
Melhor individuo: [0.99567405 1.9770381 ]
Fx para esse individuo: 1.5848931924611134


<h3>Melhores Indivíduos de cada Execução</h3>

In [99]:
bestsIwithC3

array([[ 1.76808107e-01, -3.05646086e+00],
       [ 8.92090566e-03, -3.93018744e+00],
       [-9.31114004e-02,  2.18340687e+00],
       [-8.51743859e-01,  1.25502462e+00],
       [ 1.05863179e+00, -9.04020905e-01],
       [ 1.08281975e-01, -8.63263685e-02],
       [ 9.13238380e-01, -1.98365604e+00],
       [ 9.77914081e-01, -1.00008948e+00],
       [-1.21479884e+00, -1.89355284e+00],
       [ 3.75554814e-02, -1.95996110e+00],
       [ 1.19941358e+00,  1.88492250e+00],
       [ 2.23655518e-01, -2.00356782e+00],
       [ 1.98196475e+00,  9.98298567e-01],
       [-1.04361951e+00,  2.02016217e+00],
       [ 2.92552733e+00, -3.06854156e+00],
       [-8.95034771e-01,  1.04086134e+00],
       [ 1.61917097e-01,  1.06671639e+00],
       [-9.49316405e-01, -2.47231935e-01],
       [ 2.32824524e-02, -1.02351775e+00],
       [ 1.00184959e+00,  9.75041612e-01],
       [ 1.49326352e-01, -9.13323455e-02],
       [-8.10338988e-04,  3.15452791e+00],
       [-1.03846999e+00,  1.96588689e+00],
       [ 1.

<h3>Melhores Indivíduos de cada Execução</h3>

In [100]:
bestsFxwithC3

array([[1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319]])

<h1>Comparação dos resultados</h1>

<p>Para verificarmos qual das funções de cruzamento obtive o melhor desempenho, iremos comparar a contagem de indivíduos que convergiram dentre as 30 execuções. Utilizamos como critério de convergência ambas as características do individuo atingirem valores menores que 0.1</p>

In [103]:
cont1 = 0
max1 = 0
cont2 = 0
max2 = 0
cont3 = 0
max3 = 0

for i in range(30):
    if(bestsIwithC1[i,0] < 0.1 and bestsIwithC1[i,1] < 0.1):
        cont1 = cont1 + 1
    if(bestsIwithC1[i,0] > bestsIwithC1[max1,0] and bestsIwithC1[i,1] > bestsIwithC1[max1,1]):
        max1 = i
        
    if(bestsIwithC2[i,0] < 0.1 and bestsIwithC2[i,1] < 0.1):
        cont2 = cont2 + 1
    if(bestsIwithC2[i,0] > bestsIwithC2[max2,0] and bestsIwithC2[i,1] > bestsIwithC2[max2,1]):
        max2 = i
        
    if(bestsIwithC3[i,0] < 0.1 and bestsIwithC3[i,1] < 0.1):
        cont3 = cont3 + 1
    if(bestsIwithC3[i,0] > bestsIwithC3[max3,0] and bestsIwithC3[i,1] > bestsIwithC3[max3,1]):
        max3 = i
        
print("Quantidade de indivíduos que convergiram com o o cruzamento 1: %d" %(cont1))
print("Indivíduo de pior convergência: ")
print(bestsIwithC1[max1])

print("Quantidade de indivíduos que convergiram com o o cruzamento 2: %d" %(cont2))
print("Indivíduo de pior convergência: ")
print(bestsIwithC2[max2])

print("Quantidade de indivíduos que convergiram com o o cruzamento 3: %d" %(cont3))
print("Indivíduo de pior convergência: ")
print(bestsIwithC3[max3])

Quantidade de indivíduos que convergiram com o o cruzamento 1: 16
Indivíduo de pior convergência: 
[0.98188183 1.99821593]
Quantidade de indivíduos que convergiram com o o cruzamento 2: 15
Indivíduo de pior convergência: 
[0.01468203 2.99615165]
Quantidade de indivíduos que convergiram com o o cruzamento 3: 6
Indivíduo de pior convergência: 
[1.19941358 1.8849225 ]


<h1>Conclusões</h1>

<p>Para as funções de cruzamento 1 e 2, a convergência ocorreu em cerca de 50% das 30 execuções. Já a função de cruzamento 3 convergiu apenas para 20% das 30 execuções. Apesar da terceira função de cruzamente obter o pior resultado, o indivíduo com menor convergência das 30 execuções, resultou do uso do cruzamento 2.</p>